In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn.functional import one_hot
import torch_geometric as pyg
import networkx as nx
import torch.nn.functional as F
from torch_geometric.datasets import ZINC
from torch_geometric.utils import to_networkx
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt

/home/diego/miniconda3/envs/ter_gnn/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from gcn_model import GCNNet, GCN

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
train_ds = ZINC(root='./data/zinc/', subset=True, split='train')
test_ds = ZINC(root='./data/zinc/', subset=True, split='test')

In [5]:
len(train_ds)

10000

In [6]:
len(test_ds)

1000

In [7]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [15]:
MODEL_PARAMS = {
    'GCN': {
        'channels': [
            21,
            16,
            16
        ]
    },
    'MLP': {
        'channels': [
            16,
            10,
            1
        ]
    }
}

In [16]:
model = GCNNet(MODEL_PARAMS).to(device)

In [17]:
for params in model.parameters():
    print(params.shape)

torch.Size([16])
torch.Size([16, 21])
torch.Size([16])
torch.Size([16, 16])
torch.Size([10, 16])
torch.Size([10])
torch.Size([1, 10])
torch.Size([1])


In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)

In [19]:
loss = nn.MSELoss()

In [20]:
def train(model, train_loader, optimizer, loss):
    model.train()
    loss_acc = 0
    total_graphs = 0
    for graph_batch in train_loader:
        graph_batch = graph_batch.to(device)
        optimizer.zero_grad()
        
        x_oh = one_hot(graph_batch.x.flatten(), num_classes=21).type(torch.cuda.FloatTensor)
        preds = model(x_oh, graph_batch.edge_index, graph_batch.batch).squeeze()
        loss_val = loss(preds, graph_batch.y)
        loss_acc += loss_val.item()
        total_graphs += graph_batch.num_graphs
        loss_val.backward()
        optimizer.step()
        
    loss_acc /= total_graphs
    return loss_acc

In [21]:
from sklearn.metrics import r2_score

def validate(model, valid_loader, loss):
    model.eval()
    loss_acc = 0
    total_graphs = 0
    total_preds = []
    labels = []
    with torch.no_grad():
        for graph_batch in valid_loader:
            graph_batch = graph_batch.to(device)
            x_oh = one_hot(graph_batch.x.flatten(), num_classes=21).type(torch.cuda.FloatTensor)
            preds = model(x_oh, graph_batch.edge_index, graph_batch.batch).squeeze()
            loss_val = loss(preds, graph_batch.y)
            loss_acc += loss_val.item()
            total_graphs += graph_batch.num_graphs
            total_preds.extend(preds.cpu().numpy())
            labels.extend(graph_batch.y.cpu().numpy())

    r2 = r2_score(labels, total_preds)            
    loss_acc /= total_graphs
    return loss_acc, r2

In [22]:
train_loss = []
test_loss = []
valid_accuracy = []
for epoch in range(3):
    print('EPOCH:', epoch+1)
    print('Training...')
    loss_value = train(model, train_loader, optimizer, loss)
    train_loss.append(loss_value)
    print('Training Loss:', loss_value)

    print('Validating')
    loss_value, accuracy = validate(model, test_loader, loss)
    test_loss.append(loss_value)
    valid_accuracy.append(accuracy.item())
    print('Validation Loss:', loss_value)
    print('Accuracy:', accuracy.item())


EPOCH: 1
Training...
Training Loss: 0.12742989996671678
Validating
Validation Loss: 0.13162272906303404
Accuracy: -0.004510924093766633
EPOCH: 2
Training...
Training Loss: 0.1267676159620285
Validating
Validation Loss: 0.1310012185573578
Accuracy: 0.0006444466647653746
EPOCH: 3
Training...
Training Loss: 0.12625205079317092
Validating
Validation Loss: 0.13028756546974182
Accuracy: 0.006532424928695213
